In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

In [3]:
import os
import torch

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [22]:
from pprint import pprint

from rlvsil.diversity import DEFAULT_CONFIGS, calculate_diversity_metrics

diversity_metrics = ["ead_averaged_distinct_ngrams","nli_sample_from_sim","sent_bert_from_sim"]
no_overall_input = True
sample_overall = True
no_per_input = False

outputs = ...

diversity_metrics_config = DEFAULT_CONFIGS.copy()
if diversity_metrics != "all":
    diversity_metrics_config = {
        k: v for k, v in diversity_metrics_config.items() if k in diversity_metrics
    }
print(diversity_metrics_config)

run_results = calculate_diversity_metrics(
    outputs, diversity_metrics_config, no_per_input, no_overall_input, sample_overall
)
pprint(run_results)

SyntaxError: invalid syntax (196523630.py, line 18)

In [ ]:
# resp_set = ["bla bla", "daj wiersz", "zaszczekaj glosno i dlugo"]

from rlvsil.diversity.diversity_metrics import CosineSimilarity2Diversity, AveragedCosineSimilarity, AveragedDistinctNgrams, DistinctNgrams


def print_metric(metric, resp_set):
    print("{0}: {1:0.3f}".format(type(metric).__name__, metric(resp_set)))

# TEST
resp_set = ["i am going", "i am going", "lets go i i"]
config = {"n": 3}
print_metric(CosineSimilarity2Diversity(config), resp_set)
print_metric(DistinctNgrams(config), resp_set)

avg_config = {"n_min": 1, "n_max": 5}
print_metric(AveragedCosineSimilarity(avg_config), resp_set)
print_metric(AveragedDistinctNgrams(avg_config), resp_set)


CosineSimilarity2Diversity: 0.667
DistinctNgrams: 0.750
AveragedCosineSimilarity: 0.737
AveragedDistinctNgrams: 0.593


In [ ]:
from rlvsil.diversity import DEFAULT_CONFIGS, calculate_diversity_metrics


# metrics from paper: ead_averaged_distinct_ngrams, nli_sample_from_sim, sent_bert_from_sim


outputss = [
    [
        "I like to eat apples.",
        "I like to eat bananas.",
        "I like to eat oranges.",
    ],
    [
        "I love to eat apples.",
        "I love to eat bananas.",
        "I love to eat oranges.",
    ],
    [
        "I love muching on apples.",
        "I love muching on bananas.",
        "I love muching on oranges.",
    ],
]
config = DEFAULT_CONFIGS.copy()
del config["openai_from_sim"]

from pprint import pprint

pprint(config)


In [24]:
results = calculate_diversity_metrics(outputss, metric_configs=config)

/home/djaniak/mambaforge/envs/rlhf-gen-div/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

calculating per-input diversities
Average per-input diversities:
{'mean_per_input_averaged_distinct_ngrams': 0.6377777777777778, 'mean_per_input_ead_averaged_distinct_ngrams': 0.6378211785884894, 'mean_per_input_cosine_similarity_2d_diversity': 0.3333333333333333, 'mean_per_input_sent_bert_from_sim': 0.40668171644210815, 'mean_per_input_nli_from_sim': -0.978046695391337, 'mean_per_input_nli_sample_from_sim': 0.9884730033079783}
Std per-input diversities:
{'std_per_input_averaged_distinct_ngrams': 0.0, 'std_per_input_ead_averaged_distinct_ngrams': 0.0, 'std_per_input_cosine_similarity_2d_diversity': 5.551115123125783e-17, 'std_per_input_sent_bert_from_sim': 0.0597790032252103, 'std_per_input_nli_from_sim': 0.00554125843101213, 'std_per_input_nli_sample_from_sim': 0.061663088363215636}
calculating overall diversities
Average overall diversities:
{'overall_averaged_distinct_ngrams': 0.5388888888888889, 'overall_ead_averaged_distinct_ngrams': 0.538994964957481, 'overall_cosine_similarity_2

In [ ]:
from pprint import pprint

pprint(results)


{'mean_per_input_averaged_distinct_ngrams': 0.6377777777777778,
 'mean_per_input_cosine_similarity_2d_diversity': 0.3333333333333333,
 'mean_per_input_ead_averaged_distinct_ngrams': 0.6378211785884894,
 'mean_per_input_nli_from_sim': -0.978046695391337,
 'mean_per_input_nli_sample_from_sim': 0.8935476626290214,
 'mean_per_input_sent_bert_from_sim': 0.40668171644210815,
 'overall_averaged_distinct_ngrams': 0.5388888888888889,
 'overall_cosine_similarity_2d_diversity': 0.8055555555555556,
 'overall_ead_averaged_distinct_ngrams': 0.538994964957481,
 'overall_nli_from_sim': -0.0003935595353443677,
 'overall_nli_sample_from_sim': 0.9086641861332787,
 'overall_sent_bert_from_sim': 0.3989078998565674,
 'overall_single_output_averaged_distinct_ngrams': 0.8344444444444445,
 'overall_single_output_cosine_similarity_2d_diversity': 0.888888888888889,
 'overall_single_output_ead_averaged_distinct_ngrams': 0.8345037971855376,
 'overall_single_output_nli_from_sim': 2.911579628785451,
 'overall_single

In [3]:
# !pip install transformers==4.36.2

In [7]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


model_name = "speakleash/Bielik-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)

/home/djaniak/mambaforge/envs/rlhf-gen-div/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# model_name = "speakleash/Bielik-Instruct-7B-v0.1"

In [21]:

prompt = "Pewnego razu w lesie"
N = 5
temperature = 1.0  

input_ids = tokenizer.encode(prompt, return_tensors="pt")

outputs = []
for _ in range(N):
    output = model.generate(input_ids.to(device),
                            pad_token_id=tokenizer.eos_token_id,
                            do_sample=True, 
                            max_length=50, 
                            temperature=temperature, 
                            top_k=0,
                            top_p=1
                            )
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    outputs.append(generated_text)

# Print the outputs
for i, output in enumerate(outputs):
    print(f"Output {i+1}:\n{output}\n")

Output 1:
Pewnego razu w lesie...
2015-05-23 11:50:27
Po wejściu do Ojcowskiego Parku Narodowego pierwsze co r

Output 2:
Pewnego razu w lesie nieopodal Headingstone znaleziono ciało młodego, około dwudziestopięcioletniego chłopaka. Dwó

Output 3:
Pewnego razu w lesie Entów, mieliśmy okazję poznać jednego z nich, Drzewca. W opowieści tej były wspomniane Entowe żony

Output 4:
Pewnego razu w lesie... Siedzący nad jeziorem leśnik obserwuje łódkę geologów zbierających próbki na dnie, korzystają

Output 5:
Pewnego razu w lesie zgasło Słońce i robiło się coraz zimniej. Leśne zwierzęta zaczęły się coraz bardziej denerwować

